# Welcome to better ebird API 🐦

This tutorial allows the user to explore and use [eBird API 2.0](https://documenter.getpostman.com/view/664302/S1ENwy59), request for this API can be found [here](https://ebird.org/data/download). The functions provided here are made for all those who are intrested in exploring the birds of the world as the species_data later defined and the eBird API cover all species of birds and all locations! Since ebird relies on user inputed data, therefore the mapping functions provided in this tutorial may present more observations in some areas and less in others.  

If you prefer to use these functions by importing the better_ebirdAPI module, see the "Using_better_ebirdAPI_Module.ipynb" 

I hope you enjoy and find out more about the interesting species around you! 

In [ ]:
# import and install the following 
import pandas as pd
import geopandas as gpd
import requests

In [ ]:
# Define all necessary API and CX keys
# if you dont have access to google API that is okay, you just wont be able to use the functions coded with 'GAPI'
google_apikey = "paste your google api key here"
google_CX = "paste your google CX here"
ebird_api_key = 'paste your ebird api key here'

First import the csv file with all the information on world birds.

In [ ]:
# read the following csv file. this is necessary for the following fuctions: com_to_sci, sci_to_com, list_types
species_data = pd.read_csv("PFW-species-translation-table.csv")

In [ ]:
# feel free to explore the data 
species_data

## Now that we have the data we can define and run functions based off of it! 🦆

This first function can be used to retive the scientific name of the bird using the common name as the input

In [ ]:
def com_to_sci(common_name):
    # make it not case sensitve 
    species_data = pd.read_csv("PFW-species-translation-table.csv")
    common_name = common_name.lower()
    scientific_name = species_data.loc[species_data['american_english_name'].str.lower() == common_name, 'scientific_name'].values[0]
   # this was added because for some reason sometimes the mapping functions wouldnt work without it. dont ask 
    if len(scientific_name) == 0:
        return None
    else:
        return scientific_name

In [ ]:
# test it out by inputting any common bird name into the code such as "house wren"
com_to_sci('house wren')

In [ ]:
# try it yourself! (remove # from the next line)
# com_to_sci("instert common bird name here")

This next function is the reverse of **com_to_sci**, It can be used to retive the common name of the bird using the scientific name as the input

In [ ]:
def sci_to_com(scientific_name):
    species_data = pd.read_csv("PFW-species-translation-table.csv")
    scientific_name = scientific_name.lower()
    common_name = species_data.loc[species_data['scientific_name'].str.lower() == scientific_name, 'american_english_name'].values[0]
    if len(common_name) == 0:
        return None
    else:
        return common_name

In [ ]:
# test it out by inputting any scientific bird name into the code such as "Troglodytes aedon"
sci_to_com('Troglodytes aedon')

In [ ]:
# try it yourself! (remove # from the next line)
# sci_to_com("instert common bird name here")

The third function **list_types** finds matching names and returns a list of all the speices that contain a specified word such as "duck" or "wren" 

In [ ]:
def list_types(keyword):
    matching_names = species_data[species_data['american_english_name'].str.contains(keyword, case=False)]['american_english_name'].tolist()
    return matching_names

In [ ]:
# test it out by inputting any type of bird such as robin
list_types("hummingbird")

## Using Beautiful Soup to scrape websites 🦃

the following functions use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) this python libary is used pulls information from html websites.

To use beauiful soup, you must locate the infomration you want to pull using "inspect" to find the div name

In [ ]:
# if its not already installed use !pip

In [ ]:
#import the following:
import bs4
from bs4 import BeautifulSoup
import requests

The first function will pull information from [The Audubon Bird Guide](https://www.audubon.org/bird-guide). This guide has access to all the birds in North America

The website url is set up like this 
https://www.audubon.org/field-guide/bird/aberts-towhee

As you can see from the link above, the common name of the bird is at the end and any spaces in the name are replaced with a '-'. 

In [ ]:
def com_des(common_name):
    # we must replace all spaces with a '-' for the function to work
    common_name = common_name.replace(" ", "-")
    # we know the common name goes at the end based off of exploring the website
    url = f"https://www.audubon.org/field-guide/bird/{common_name}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # by inspecting the website we know where the desription of the bird is located
    bird_div = soup.find("div", class_="bird_discussion")
    if bird_div is not None:
        return bird_div.text.strip()
    else:
        return "Bird not found on Audubon Bird Guide. The guide only has access to birds of North America. If the bird is in North America, check spelling"


In [ ]:
# test it out 
com_des("mallard")

#### What if we want to find a bird outside of North America? 

The following function pulls information from [picturebirdai.com/](https://picturebirdai.com/) this website has access to all the birds of the world! 

The website url is set up like this: https://picturebirdai.com/wiki/Amazilia_yucatanensis.html

As we can see from the link above, the scientific name is at the and and any space in the name are replace with a "_"

In [ ]:
def sci_des(scientific_name):
    # we must replace all spaces with a '_' for the function to work
    scientific_name = scientific_name.replace(" ", "_")
     # we know the scientific name is inputted at the end and followed by a ".html" by exploring the website 
    url = f"https://picturebirdai.com/wiki/{scientific_name}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # by inspecting the website we know where the desription of the bird is located
    bird_div = soup.find("div", {"id": "description-content"})
    if bird_div is not None:
        field_value_div = bird_div.find("div", {"class": "field_value_description"})
        if field_value_div is not None:
            div = field_value_div.find("div")
            if div is not None:
                return div.text.strip()
            return "Bird not found. Make sure you are inputting the proper scientific name. Use function com_to_sci to get the proper name"


In [ ]:
# test it out!
# if you  dont know the scientific name of the bird you want use the function com_to_sci 
# example
sci_des("Anas platyrhynchos")

Lets combine these functions! Lets define a function called **bird_description** which will take the common name as input and search [The Audubon Bird Guide](https://www.audubon.org/bird-guide), if nothing is found it will then search [picturebirdai.com](https://picturebirdai.com/). 

This function will use **com_to_sci** to convert the common name in order it to be searched in **picturebirdai.com** if it isnt found in **The Audubon Bird Guide**

In [ ]:
def bird_description(common_name):
    # the next line is there as in the mapping functions, bird_description failed to return any birds which end with (common) 
    # This gits rid of anything in parentheses at the end of a name. 
    #sometimes in species_data the bird end with its regional subspecies such as Orange-crowned Warbler (celata) or  
    # Orange-crowned Warbler (lutescens), unfortantely there is typically only one page dedicated to the species (Orange-crown warbler)
    # and therefore the descrpition may be inaccurate to the subspeices. 
    common_name = common_name.split(' (')[0] 
    common_name1 = common_name.replace(" ", "-")
    url = f"https://www.audubon.org/field-guide/bird/{common_name1}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    bird_div = soup.find("div", class_="bird_discussion")
    if bird_div is not None:
        return bird_div.text.strip()
    else:
        scientific_name = com_to_sci(common_name)
        if scientific_name is not None:
            scientific_name1 = scientific_name.replace(" ", "_")
            url = f"https://picturebirdai.com/wiki/{scientific_name1}.html"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            bird_div = soup.find("div", {"id": "description-content"})
            if bird_div is not None:
                field_value_div = bird_div.find("div", {"class": "field_value_description"})
                if field_value_div is not None:
                    div = field_value_div.find("div")
                    if div is not None:
                        return div.text.strip()
            return "Bird not found. Make sure you are inputting the proper scientific name. Use function com_to_sci to get the proper name"
        else:
            return "Bird not found"

In [ ]:
# test it out!
bird_description("common_name")

# Explore the ebird API! 🦢 

This function retieves the data from the ebird API about a specifed species using the scientfic name as the input

In [ ]:
# if you already defined your api key above then you can remove it from your input 
# ex. retrieve_species_data(scientific_name) 
# this isnt necessary but I like it because it makes it makes using the function quicker. 
#it is  set up this way so it aligns with the .py file
def retrieve_species_data(scientific_name, ebird_api_key):
    url = "https://api.ebird.org/v2/ref/taxonomy/ebird"
    params = {
        "fmt": "json",
        "locale": "en",
        "species": scientific_name
    }
    headers = {
        "X-eBirdApiToken": ebird_api_key
    }
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    if data:
        return data[0]
    else:
        return None

In [ ]:
retrieve_species_data('scientific_name', 'ebird_api_key')

# Retiving Images! 🦅

In [ ]:
#Import the following 
import IPython 
from IPython.display import Image
import json

there are two ways to Retieve images:
1. The first way uses BeautifulSoup to find the image within an website 
2. The second way uses a Google API to do a google image search

### First way: Using BeautifulSoup and Wikipedia

In [ ]:
def wiki_pic(common_name):
    # Make a GET request to the Wikipedia page
    #it searches the scientfic name because wikipedia automattically connects the scientfic name and the common name to the same page
    # doing this ensures that the function will only return pictures of birds not other things of the same name
    sci_name = com_to_sci(common_name).replace(" ", "_")
    url = f"https://en.wikipedia.org/wiki/{sci_name}"
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the infobox table with either class name
    infobox = soup.find("table", class_=["infobox biota", "taxobox"])
    # if there is an infobox find image
    if infobox:
        # Find the image link within the infobox table
        image = infobox.find("img")
        # Get the source URL of the image
        image_url = image["src"]
        # Make a GET request to the image URL
        response = requests.get("https:" + image_url)
        picture_url = ("https:" + image_url)
    # if no infobox they are usually in a div called thmbinner
    else:
        picbox = soup.find("div", class_=["thumbinner"])
        if picbox:
            # Find the first image in the thumbinner div
            image2 = picbox.find("img")
            # Get the source URL of the image
            image_url = image2["src"]
            # Make a GET request to the image URL
            response = requests.get("https:" + image_url)
            picture_url = ("https:" + image_url)
        else:
            return "image not found"  
    #return(picture_url)
    display(Image(url=picture_url))

This functions converts the common name of the bird to its scientfic name to ensure it returns the correct image, however by doing so it wont return anything unless the correct common name is inputed as it must align with a name in species data. 

if you are unsure its true name, use list_types. (ex. it wont return an image of a 'swan' but will return an image of a 'trumpter swan' or a 'tundra swan')

In [ ]:
#Try it
wiki_pic('common_name')

### Second way: Using Google Image Search

In [ ]:
# import the following 
from google_images_search import GoogleImagesSearch

In [ ]:
def google_API_pic(bird_name):
    # Initialize GoogleImagesSearch object
    gis = GoogleImagesSearch(google_apikey, google_CX)
    # Set search parameters
    _search_params = {
        # + bird is added to ensure that only bird images get retieved or else you would retieve TV images if merlin is search or sports team logos etc
        "q": bird_name + " bird",
        "imgSize": "large",
        "num": 1
    }
    # Search for images
    gis.search(search_params=_search_params)
    # Get image URL
    image_url = gis.results()[0].url
    pic = requests.get(image_url)
    # Display image in notebook
    return Image(pic.content)

# Encode the images for mapping! 🦉


We are soon going to define some mapping functions however, we will want to use these functions to import the images of the birds into the map markers/popups. 

In order to do so we must encode the images. This will use [base64](https://docs.python.org/3/library/base64.html)

In [ ]:
#import the following 
import base64

## For Wiki images 

In [ ]:
def wiki_pic_map(common_name):
    # Make a GET request to the Wikipedia page
    sci_name = com_to_sci(common_name)
    url = f"https://en.wikipedia.org/wiki/{sci_name}"
    common_name = common_name.replace(" ", "_")
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    # Find the infobox table with either class name
    infobox = soup.find("table", class_=["infobox biota", "taxobox"])
    # Find the first image in the infobox table, if it exists
    if infobox:
        # Find the image link within the infobox table
        image = infobox.find("img")
        # Get the URL of the image
        image_url = image["src"]
        response = requests.get("https:" + image_url)
        # Encode the image content as base64
        image_data = base64.b64encode(response.content).decode()
        # Returnimage data
        return image_data
    # If there is no image in the infobox table, look for the first image in the thumbinner div
    else:
        picbox = soup.find("div", class_=["thumbinner"])
        if picbox:
            # Find the first image in the thumbinner div
            image2 = picbox.find("img")
            # Get the URL of the image
            image_url = image2["src"]
            response = requests.get("https:" + image_url)
            # Encode the image content as base64
            image_data = base64.b64encode(response.content).decode()
            # Return image data
            return image_data
        # If no image is found, return None
        else:
            return None 

## For Google images 

In [ ]:
def google_API_pic4map(bird_name, google_apikey, google_CX):
    # Initialize GoogleImagesSearch object
    gis = GoogleImagesSearch(google_apikey, google_CX)

    # Set search parameters
    _search_params = {
        "q": bird_name + " bird",
        "imgSize": "large",
        "num": 1
    }

    # Search for images
    gis.search(search_params=_search_params)
    image_data = requests.get(gis.results()[0].url).content
    # Encode image data as base64
    image_base64 = base64.b64encode(image_data).decode()
    
    return image_base64

# Lets Map! 🦜

In [ ]:
#Import the following
import folium

For our maps we will use some of the functions defined earlier in this lab. 

- **bird_description** will be used to pull a descrption of the bird for the marker popup 
- **wiki_pic_map** or **google_API_pic4map** will be used to retieve a picture of the mapped bird for the marker popup

(The speed of these functions depend on the amount of birds within your defined radius, I suggest working with radius of under 10km unless you are are a patient person)

#### Reasoning for the Google API Versions: 
The google API versions were the orginally created functions, however there are barriers for using them as they require the user to enable [Custom Search JSON API](https://developers.google.com/custom-search/v1/overview) on their [Google Cloud Console](https://console.cloud.google.com/) and for the user to have a Google API key and Google CX. While these are easy to retrieve and are free there is a limit to the amount of queries per user (100) unless the user signs up for billing ($5 for every additonal 1000 queries). To address this problem, the wiki_pic fucntions were created to retieve the photos for the maps in its place. 

The Google API version has been more consitstant in loading all the bird images within the popup windows, and I would encorage those who have access to an increased number of queries to use it or to reduce the radius in your search in order to maximize the use of the API. 

## Our first function will find ***nearby birds*** 🐧

- this function takes the coordinates of any location and a radius distance

In [ ]:
# if already defined your ebird_api_key key above you can get rid of them here 
#and only change it to "nearby_observations(lat, lng, distance_km)" 
# it is set up this way so it aligns with the .py file This goes for any of the mapping functions
def nearby_observations(lat, lng, distance_km, ebird_api_key):
    # Define the API endpoint URL
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist={distance_km}&cat=species&fmt=json'
    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)

    data = response.json()
    
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    # Loop through the data and add markers for each bird sighting
    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        popup_text = f"<b>{bird_name}</b><br>{sci_name}</br><br>Date:{obs_date}</br>"
            
 # Get bird description
        bird_description_text = bird_description(bird_name)
 
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}" style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map




In [ ]:
nearby_observations('lat', 'lng', 'distance_km', 'ebird_api_key')

### Google Version of Nearby Observations 

In [ ]:
# if already defined your ebird_api_key, google_apikey and google_CX keys above you can get rid of them here 
#and only change it to "nearby_observations_GAPI(lat, lng, distance_km)" 
# its set up this way so it aligns with the .py file This goes for any of the _GAPI functions
def nearby_observations_GAPI(lat, lng, distance_km, ebird_api_key, google_apikey, google_CX):
    # Define the API endpoint URL
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist={distance_km}&cat=species&fmt=json'
    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)

    data = response.json()
    
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    # Loop through the data and add markers for each bird sighting
    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        popup_text = f"<b>{bird_name}</b><br>{sci_name}</br><br>Date:{obs_date}</br>"
            
 # Get bird description
        bird_description_text = bird_description(bird_name)
        #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        bird_image_data = google_API_pic4map(bird_name, google_apikey, google_CX)
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}" style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map


In [ ]:
nearby_observations_GAPI('lat', 'lng', 'distance_km', 'ebird_api_key', 'google_apikey', 'google_CX')

## Our next function will find ***nearby sighting of a specific species*** 🦚

- this function takes the coordinates of any location and the birds scientific name 
- it has a default radius of 10km

In [ ]:
def nearby_bird(lat, lng, sci_name, ebird_api_key):
    # Define the API endpoint URL
    #dis = 10km
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist=10&sci={sci_name}&fmt=json'
    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)
    data = response.json()

    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
         # this function defaults to using wiki_pic_map however it can be replace with google_API_pic4map. 
        #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        # to use the function replace the next line of code with " bird_image_data = google_API_pic4map(bird_name)
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map

In [ ]:
# try it 
nearby_bird('lat', 'lng', 'sci_name', 'ebird_api_key')

### Google Version of Nearby Bird

In [ ]:
def nearby_bird_GAPI(lat, lng, sci_name, ebird_api_key, google_apikey, google_CX):
    # Define the API endpoint URL
    #dis = 10km
    url = f'https://api.ebird.org/v2/data/obs/geo/recent?lat={lat}&lng={lng}&dist=10&sci={sci_name}&fmt=json'

    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)
    data = response.json()

    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
         
        #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        bird_image_data = google_API_pic4map(bird_name, google_apikey, google_CX)
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map

In [ ]:
# try it 
nearby_bird_GAPI('lat', 'lng', 'sci_name', 'ebird_api_key', 'google_apikey', 'google_CX')

## Our next function will find ***nearby sighting of notable species*** 🐥
- this function takes the coordinates of any location and a specified radius in km

In [ ]:
def notable_bird(lat, lng, distance_km, ebird_api_key):
    # Define the API endpoint URL
  
    url =f'https://api.ebird.org/v2/data/obs/geo/recent/notable?lat={lat}&lng={lng}&dist={distance_km}&fmt=json'
    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}
    # Send GET request to API endpoint
    response = requests.get(url, headers=headers)

    # Convert response to JSON format
    data = response.json()

    # Create a map centered at the inputted lat/lng
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
        #if bird_description_text == "Bird not found on Audubon Bird Guide. The guide only has access to birds of North America. If the bird is in North America, check spelling":
          #  bird_description_text = sci_des(sci_name)
        bird_image_data = wiki_pic_map(bird_name)
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map


In [ ]:
# try it 
notable_bird('lat', 'lng', 'distance_km', 'ebird_api_key')

### Google Version of Notable Bird

In [ ]:
def notable_bird_GAPI(lat, lng, distance_km, ebird_api_key, google_apikey, google_CX):
    # Define the API endpoint URL
  
    url =f'https://api.ebird.org/v2/data/obs/geo/recent/notable?lat={lat}&lng={lng}&dist={distance_km}&fmt=json'

    # Add API key to the request headers
    headers = {'X-eBirdApiToken': ebird_api_key}
    # Send GET request to API endpoint
    response = requests.get(url, headers=headers)

    # Convert response to JSON format 
    data = response.json()

    # Create a map centered at the inputted lat/lng
    bird_map = folium.Map(location=[lat, lng], zoom_start=10)

    for obs in data:
        bird_name = obs['comName']
        sci_name = obs['sciName']
        obs_date = obs['obsDt']
        location = [obs['lat'], obs['lng']]
        
        # Get bird description
        bird_description_text = bird_description(bird_name)
        #if bird_description_text == "Bird not found on Audubon Bird Guide. The guide only has access to birds of North America. If the bird is in North America, check spelling":
          #  bird_description_text = sci_des(sci_name)
         #google_API_pic4map is better if the user doesnt have a limit in querys as it is more reliable 
        bird_image_data = google_API_pic4map(bird_name, google_apikey, google_CX)
        
        
        # Create the HTML popup
        popup_html = f"""
        <h4>{bird_name}</h4>
        <img src="data:image/jpeg;base64,{bird_image_data}"style="max-width:100%;max-height:100%">
        <h5><p><b>Scientific name:</b> {sci_name}</p></h5>
        <p><b>Date:</b> {obs_date}</p>
        <p><b>Description:</b> {bird_description_text}</p>
        """
        
        marker = folium.Marker(location=location, popup=folium.Popup(popup_html, max_width=400))
        marker.add_to(bird_map)

    return bird_map


In [ ]:
# try it 
notable_bird_GAPI('lat', 'lng', 'distance_km', 'ebird_api_key', 'google_apikey', 'google_CX')

## Our last mapping function will find ***nearby birding hotspots*** 🦩

- Hotspots represent a set of public locations that people regularly visit for birding, regardless of how amazing they are for birds. The primary requirement of a Hotspot is that it is publicly accessible
- this function is great for those wondering where others frequently birdwatch at

In [ ]:
def get_hotspot(lat, lng, ebird_api_key):
    url = f'https://api.ebird.org/v2/ref/hotspot/geo?lat={lat}&lng={lng}&dist=5&fmt=json'
    headers = {'X-eBirdApiToken': ebird_api_key}
    response = requests.get(url, headers=headers)
    data = response.json()
    bird_map = folium.Map(location=[lat, lng], zoom_start=13)

    for obs in data:
        loc_id = obs['locId']
        #then use the api to go find the hotspot information so it can be put into the markers 
        hotspot_url = f"https://api.ebird.org/v2/ref/hotspot/info/{loc_id}"
        hotspot_response = requests.get(hotspot_url, headers=headers)
        hotspot_data = hotspot_response.json()
        if hotspot_response.status_code == 200:
            # Create popup text with hotspot info
            popup_text = f"<b> Name: </b><br>{hotspot_data['name']}</br>" \
                         f"<b>Region:</b><br>{hotspot_data['subnational2Name']}</br>" \
                         f"<b>Province/State:</b><br>{hotspot_data['subnational1Name']}</br>" \
                         f"<b>Country:</b><br>{hotspot_data['countryCode']}</br>" \
                         f"<b>Latitude & Longitude:</b><br>{hotspot_data['latitude']}, {hotspot_data['longitude']}</br>"\
            
        else:
            popup_text = "Error getting hotspot info"
            location = obs['lat'], obs['lng']
            marker = folium.Marker(location=location, popup=popup_text)
            marker.add_to(bird_map)

            
    return bird_map

In [ ]:
# try it 
get_hotspot('lat', 'lng', 'ebird_api_key')